# Visualise Plots

In [1]:
import pdal
import pandas as pd
import numpy as np
import geopandas as gpd

import pyvista as pv
pv.set_jupyter_backend('client')

from pathlib import Path
import ipywidgets as widgets
from IPython.display import display

In [2]:
plot_lidar_dir = Path("../data/outputs/plots/lidar")
plots_gdf = gpd.read_file("../data/outputs/plots/plots.geojson")
# plots_gdf
plots = plots_gdf["id"].to_list()

In [7]:
def read_plot(plot_id: str):
    plot_path = plot_lidar_dir / f"{plot_id}.copc.laz"
    pl = pdal.Reader(str(plot_path)).pipeline()
    pl.execute()
    return pl.arrays[0]

def render_point_cloud(points, color='rgb'):
    X = points["X"] - points["X"].min()
    Y = points["Y"] - points["Y"].min()
    Z = points["Z"]
    positions = np.column_stack((X, Y, Z)).astype("float32")

    R = (points["Red"] / 65535 * 255).astype("uint8")
    G = (points["Green"] / 65535 * 255).astype("uint8")
    B = (points["Blue"] / 65535 * 255).astype("uint8")
    colors = np.column_stack((R, G, B))

    mesh = pv.PolyData(positions)
    mesh['height'] = Z
    mesh['rgb'] = colors
    mesh.plot(point_size=3, scalars=color, rgb=(color == 'rgb'), notebook=True, cmap='inferno')

def read_and_render_plot(plot_id: str, color='rgb'):
    points = read_plot(plot_id)
    render_point_cloud(points, color=color)

In [ ]:
# Create interactive widgets for site selection
plot_dropdown = widgets.Dropdown(
    options=plots,
    value=plots[0] if plots else None,
    description='plot:',
    disabled=False,
)

color_dropdown = widgets.Dropdown(
    options=['rgb', 'height'],
    value='rgb',
    description='Color by:',
    disabled=False,
)

# Function to handle the plot update
def update_plot(plot, color):
    read_and_render_plot(plot, color=color)

# Create interactive plot
interactive_plot = widgets.interactive(update_plot, plot=plot_dropdown, color=color_dropdown)
display(interactive_plot)

interactive(children=(Dropdown(description='plot:', options=('AGG_O_01_P1', 'AGG_O_01_P2', 'AGG_O_01_P3', 'AGG…